In [2]:
import pandas as pd

In [193]:
#municipality = pd.read_csv("clean_data/turkey_municipality.csv")
#parlimentary = pd.read_csv("clean_data/turkish_parlimentary.csv")
#parlimentary_d = pd.read_csv("clean_data/milletvekili_detailed.csv")
party_alignment = pd.read_excel("clean_data/party_alignment.xlsx")
#events_raw = pd.read_excel("data/turkish_catastrophic_event.xlsx")

In [89]:
parlimentary = parlimentary.sort_values(by = ["year", "turnout"], ascending = False)
parlimentary = parlimentary.drop_duplicates(subset = ["year"], keep = "first")

# Municipality

In [107]:
municipality.drop(columns = ['prev_year',
       'winner', "prev_winner", "swing"], inplace = True)
municipality.city = municipality.city.str.replace("mersin", "icel")
municipality = municipality[(municipality.city != "turkiye") & (municipality.city != "altkademebelediyesi")]
municipality["percent_turnout"] = municipality["turnout"] / municipality["registered_voters"]

In [23]:
def determine_winner(row):
    row = row.dropna(axis=0, how='any')
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [111]:
years = list(municipality.year.unique())
years = sorted(years, reverse = True)
municipality["prev_year"] = municipality.year.apply(lambda row: None if row == 1989 else years[years.index(row) + 1])

In [112]:
municipality["winner"] = municipality.apply(lambda row: determine_winner(row), axis = 1)
municipality["winner"] = municipality["winner"].str.split("_", expand = True)[0]

In [115]:
municipality_results = municipality[["city", "year", "winner"]].drop_duplicates()
municipality_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
municipality = municipality.merge(municipality_results, on = ["city", "prev_year"], how = "left")

In [116]:
municipality["swing"] = ((municipality["winner"] != municipality["prev_winner"]) & (municipality["prev_winner"].notna())).astype(int)

In [117]:
municipality.winner = municipality.winner.str.lower()
municipality.prev_winner = municipality.prev_winner.str.lower()
municipality.winner = municipality.winner.str.replace("ak parti", "akp")
municipality.prev_winner = municipality.prev_winner.str.replace("ak parti", "akp")

In [118]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
party_alignment.winner = party_alignment.winner.str.replace("ak parti", "akp")
municipality = municipality.merge(party_alignment, on = "winner", how = "left")

In [119]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]

In [121]:
municipality = municipality.merge(party_alignment, on = "prev_winner", how = "left")
municipality.to_csv("clean_data/turkey_municipality.csv", index = False)

# Parlimentary

In [90]:
parlimentary.drop(columns = ["num_milletvekili", "toplam_gecerli_oy_sayisi", "milletvekili_sayisi", "bgmztotal_votes"], inplace = True)
parlimentary.rename(columns = {"registered_voter": "registered_voters",
                             'valid_votes': "valid_votes"}, inplace = True)
cols = [i for i in parlimentary.columns if "percent_votes" not in i]
parlimentary = parlimentary[cols]
parlimentary.columns = [i.replace("total_votes", "") for i in parlimentary.columns]
parlimentary["city"] = ["turkiye"]*len(parlimentary)

In [91]:
years = list(parlimentary.year.unique())
years = sorted(years, reverse = True)
parlimentary["prev_year"] = parlimentary.year.apply(lambda row: None if row == "1950" else years[years.index(row) + 1])

In [92]:
def determine_winner_p(row):
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    row = row.dropna(axis=0, how='any')
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [93]:
parlimentary["winner"] = parlimentary.apply(lambda row: determine_winner_p(row), axis = 1)
parlimentary["winner"] = parlimentary["winner"].str.split("_", expand = True)[0]
parlimentary.winner = parlimentary.winner.str.replace("ak", "akp")

In [94]:
parlimentary_results = parlimentary[["year", "winner"]].drop_duplicates()
parlimentary_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
parlimentary = parlimentary.merge(parlimentary_results, on = ["prev_year"], how = "left")
parlimentary.loc[18, "prev_winner"] = "chp"

In [95]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
parlimentary = parlimentary.merge(party_alignment, on = "winner", how = "left")

In [96]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]
parlimentary = parlimentary.merge(party_alignment, on = "prev_winner", how = "left")

In [97]:
parlimentary["swing"] = ((parlimentary["winner"] != parlimentary["prev_winner"]) & (parlimentary["prev_winner"].notna())).astype(int)

In [98]:
parlimentary.winner = parlimentary.winner.str.lower()
parlimentary.prev_winner = parlimentary.prev_winner.str.lower()
parlimentary.winner = parlimentary.winner.str.replace("ak parti", "akp")
parlimentary.prev_winner = parlimentary.prev_winner.str.replace("ak parti", "akp")

# Concat Municipality and Parlimentary

In [99]:
parlimentary["elec_type"] = ["parlimentary"] * len(parlimentary)
municipality["elec_type"] = ["municipality"] * len(municipality)

In [100]:
elections_all = pd.concat([parlimentary, municipality], ignore_index = True)

In [104]:
def determine_winner_voteshare(row):
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout", 'ytp_','prev_year',
                    'winner', 'prev_winner','nationalist', 'right_wing',
                    'left_wing', 'political_islam','kurdish','liberal',
                    'socialist','soc_democ','populist','nat_prev','right_wing_prev',
                    'left_wing_prev','political_islam_prev','kurdish_prev',
                    'liberal_prev', 'socialist_prev','social_democ_prev',
                    'populist_prev','swing','elec_type'])
    row = row.dropna(axis=0, how='any')
    row = row.astype(int)
    vote_share = max(row)
    return vote_share

In [62]:
elections_all.to_csv("clean_data/elections_all.csv", index = False)

In [76]:
elections_all = pd.read_csv("clean_data/elections_all.csv")

In [105]:
elections_all["incumbent_vote_share"] = elections_all.apply(lambda row:determine_winner_voteshare(row), axis = 1)

In [107]:
elections_all["incumbent_vote_share"] = elections_all["incumbent_vote_share"] / elections_all["valid_vote"]

In [109]:
elections_all["incumbent_vote_share"] = elections_all["incumbent_vote_share"]*100

# Events

In [ ]:
events = pd.get_dummies(events)
events.to_csv("clean_data/turkish_crises.csv", index = False)

In [147]:
events_raw.year = events_raw.year.astype(int)

In [148]:
elections_all = elections_all.merge(events_raw, on = "year", how = "left")

In [113]:
events = pd.read_csv("clean_data/turkish_crises.csv")

In [140]:
elections_all = elections_all.merge(events, on = "year", how = "left")

In [141]:
cols = ["crisis type_civil unrest", "crisis type_coup", "crisis type_economic", 
       "crisis type_refugee", "crisis type_regufee", "crisis type_soft coup", "crisis type_terrorism",
       "crisis type_war"]
for col in cols:
    elections_all[col] = elections_all[col].fillna(0)

In [142]:
col_new = ["civil_unrest", "coup", "economic", "r1", "r2", "soft_coup", "terrorism", "war"]
renames = dict(zip(cols, col_new))
elections_all.rename(columns = renames, inplace = True)

In [143]:
elections_all["refugee"] = elections_all["r1"] + elections_all["r2"]
elections_all.drop(columns = ["r1", "r2"], inplace = True)

In [149]:
elections_all.to_csv("clean_data/elections_all.csv", index = False)

# City Parlimentary Manipulate

In [194]:
parlimentary_c = pd.read_csv("clean_data/parlimentary_cities.csv")

In [195]:
parlimentary_c = parlimentary_c[parlimentary_c.registered_voters.notna()]

In [196]:
def determine_winner(row):
    row = row.drop(["city", "year", "registered_voters", 
                       "turnout", "valid_vote", 
                       "percent_turnout"])
    
    row = row.dropna(axis=0, how='any') 
    row = row.astype(int)
    winner = row.idxmax()
    return winner

In [197]:
parlimentary_c["winner"] = parlimentary_c.apply(lambda row: determine_winner(row), axis = 1)
parlimentary_c.winner = parlimentary_c.winner.str.replace("ak parti", "akp")

In [198]:
parlimentary_c.year = parlimentary_c.year.str.replace('7 Haziran 2015', "2015_2")
parlimentary_c.year = parlimentary_c.year.str.replace('1 Kasım 2015', "2015_1")
years = list(parlimentary_c.year.unique())
years = sorted(years, reverse = True)
parlimentary_c["prev_year"] = parlimentary_c.year.apply(lambda row: None if row == "1950" else years[years.index(row) + 1])

In [199]:
parlimentary_c_results = parlimentary_c[["year", "city", "winner"]].drop_duplicates()
parlimentary_c_results.rename(columns = {"winner": "prev_winner", "year": "prev_year"}, inplace = True)
parlimentary_c = parlimentary_c.merge(parlimentary_c_results, on = ["prev_year", "city"], how = "left")

In [200]:
parlimentary_c["swing"] = ((parlimentary_c["winner"] != parlimentary_c["prev_winner"]) & (parlimentary_c["prev_winner"].notna())).astype(int)

In [201]:
parlimentary_c.winner = parlimentary_c.winner.str.lower()
parlimentary_c.prev_winner = parlimentary_c.prev_winner.str.lower()
parlimentary_c.winner = parlimentary_c.winner.str.replace("ak parti", "akp")
parlimentary_c.prev_winner = parlimentary_c.prev_winner.str.replace("ak parti", "akp")

In [202]:
party_alignment.rename(columns = {"party": "winner"}, inplace = True)
party_alignment.winner = party_alignment.winner.str.replace("ak parti", "akp")
parlimentary_c = parlimentary_c.merge(party_alignment, on = "winner", how = "left")

In [203]:
party_alignment.columns = ["prev_winner", "nat_prev", "right_wing_prev", "left_wing_prev",
                           "political_islam_prev", "kurdish_prev", "liberal_prev", "socialist_prev",
                          "social_democ_prev", "populist_prev"]

In [204]:
parlimentary_c = parlimentary_c.merge(party_alignment, on = "prev_winner", how = "left")
#parlimentary_c.to_csv("clean_data/turkey_parlimentary_selectedcities.csv", index = False)

In [205]:
parlimentary_c.drop(columns = [parlimentary_c.columns[-36], parlimentary_c.columns[-37]], inplace = True)

In [207]:
parlimentary_c["election_type"] = ["parlimentary"] * len(parlimentary_c)

# Concat Detailed Parlimentary Election Data

In [208]:
municipality = pd.read_csv("clean_data/turkey_municipality.csv")

In [209]:
municipality["election_type"] = ["municipality"] * len(municipality)

In [210]:
df_final = pd.concat([municipality, parlimentary_c], ignore_index = True)

In [212]:
list8df_final.columns

Index(['city', 'year', 'registered_voters', 'turnout', 'valid_vote',
       'saadet partisi', 'btp', 'tkp', 'vatan partisi', 'chp',
       ...
       'ckmp', 'cmp', 'demokratik parti', 'gp', 'hurriyet partisi', 'msp',
       'tbp', 'tip', 'vp', 'ytp '],
      dtype='object', length=118)